# Emissionsquellen

Daten Quelle: https://www.umweltbundesamt.de/sites/default/files/medien/361/bilder/dateien/2020_thg_crf_plus_1a_details_ci.xlsx

In [1]:
%reset -f
%matplotlib agg

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Image

plt.rcParams['figure.figsize'] = (8, 6)

In [3]:
_df = pd.read_excel("2020_thg_crf_plus_1a_details_ci.xlsx", skiprows=8)
_df.head(2)

,Unnamed: 0,Unnamed: 1,Energiewirtschaft / Energy Industries,Verarbeitendes Gewerbe / Manufacturing Industries and Construction,Verkehr/Transport,Übrige Feuerungsanlagen & sonstige kleine Quellen / Other Sectors & other small sources**,Diffuse Emissionen aus Brennstoffen / Fugitive Emissions from Fuels,Industrieprozesse/Industrial Processes,Landwirtschaft/Agriculture,Abfall & Abwasser / Waste & Waste Water,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,NaN,1990.0,427.353073,186.708766,164.978128,220.311287,37.696693,96.832708,79.305198,38.301930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1991.0,413.164017,165.288102,167.961403,216.942028,36.673044,93.106233,71.545190,39.678086,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# limit to the actual data
df = _df.rename(columns={'Unnamed: 1': 'date'})
# only include valid dates
df = df.loc[(1990 <= df['date']) & (df['date'] <= 2018)]
# remove last row, which reshows the 2017 number and seems to be wrong
df.drop([len(df) - 1], inplace=True)
# removed unused coloumns
df.drop(columns=[k for k in df.keys() if 'Unnamed:' in k], inplace=True)
# use year as index and make it integer
df = df.set_index('date')
# use only the german names
df = df.rename(columns={k: k.split('/')[0].strip() for k in df.keys()})
df.index = df.index.astype('i4')
df.head(2)

,Energiewirtschaft,Verarbeitendes Gewerbe,Verkehr,Übrige Feuerungsanlagen & sonstige kleine Quellen,Diffuse Emissionen aus Brennstoffen,Industrieprozesse,Landwirtschaft,Abfall & Abwasser,Summe
1990,427.353073,186.708766,164.978128,220.311287,37.696693,96.832708,79.305198,38.301930,1251.487783
1991,413.164017,165.288102,167.961403,216.942028,36.673044,93.106233,71.545190,39.678086,1204.358104


In [5]:
# aggregate
df['Verarbeitendes Gewerbe +\nIndustrieprozesse'] = \
    df['Verarbeitendes Gewerbe'] + df['Industrieprozesse']
df['Übrige Feuerungsanlagen & sonstige kleine Quellen +\nDiffuse Emissionen aus Brennstoffen'] = \
    df['Übrige Feuerungsanlagen & sonstige kleine Quellen'] + df['Diffuse Emissionen aus Brennstoffen']

In [6]:
print(df.keys())
keys = [
    #'Industrieprozesse',
    #'Verarbeitendes Gewerbe',
    'Verarbeitendes Gewerbe +\nIndustrieprozesse',
    'Energiewirtschaft',
    'Landwirtschaft',
    'Verkehr',
    #'Diffuse Emissionen aus Brennstoffen',
    #'Übrige Feuerungsanlagen & sonstige kleine Quellen',
    'Übrige Feuerungsanlagen & sonstige kleine Quellen +\nDiffuse Emissionen aus Brennstoffen',
    #'Abfall & Abwasser',
    #'Summe'
]

Index(['Energiewirtschaft', 'Verarbeitendes Gewerbe', 'Verkehr',
       'Übrige Feuerungsanlagen & sonstige kleine Quellen',
       'Diffuse Emissionen aus Brennstoffen', 'Industrieprozesse',
       'Landwirtschaft', 'Abfall & Abwasser', 'Summe',
       'Verarbeitendes Gewerbe +\nIndustrieprozesse',
       'Übrige Feuerungsanlagen & sonstige kleine Quellen +\nDiffuse Emissionen aus Brennstoffen'],
      dtype='object')


In [7]:
28/7

4.0

In [8]:
with plt.xkcd():
    plt.rcParams['font.family'] = 'DejaVu Sans'
    plt.rcParams['font.size'] = 12
    kwargs = dict(alpha=0.3, align='center', width=0.8, stacked=False)
    fig, ax = plt.subplots(1, 1, constrained_layout=True)
    df.loc[df.index.isin(df.index[::7]), keys].plot.bar(ax=ax, **kwargs)
    ax.grid(axis='y')
    ax.axhline(163, color='darkred', ls='--', lw=3, alpha=0.3)
    ax.set_ylabel('Treibhausgas-Emissonen in Mio. t CO2-equi.')
    ax.set_ylim(0, 550)
    #ax.legend(bbox_to_anchor=(0.5, -0.15)).get_frame().set_alpha(0.4)
    ax.legend().get_frame().set_alpha(0.4)
    #plt.setp(L.texts, family='Consolas')
    plt.savefig('treibhausgase_nach_sektor.png')
    plt.close()

In [9]:
%%html
<img style='width: 600px' src="treibhausgase_nach_sektor.png"></img>

In [10]:
with plt.xkcd():
    plt.rcParams['font.family'] = 'DejaVu Sans'
    plt.rcParams['font.size'] = 12
    cumsum = df[['Summe']].cumsum()\
        .rename(columns={'Summe': 'In der Atmosphäre verbleibenden\nTreibhausgase Deutschlands seit 1990'}) \
        * (1 - 0.45) 
    #percent = 55
    #cumsum['%d%% davon verbleiben in Atmosphäre' % (percent)] = cumsum['Summe'] * percent/100
    ax = cumsum[df.index.isin(df.index[::7])].plot.bar(alpha=0.3, color='0.4')
    ax.set_ylabel('Treibhausgas-Emissonen in Mio. t CO2-equi.');
    plt.savefig('treibhausgase_kumulativ.png')
    plt.close()

In [11]:
%%html
<img style='width: 600px' src="treibhausgase_kumulativ.png"></img>